In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import requests

# دانلود تصویر از گیت‌هاب
def download_image_from_github(url, save_path='digit_image.png'):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print("✅ تصویر دانلود شد.")
    else:
        raise Exception("❌ دانلود تصویر با خطا مواجه شد.")

# پیش‌پردازش و معکوس رنگ تصویر
def preprocess_image(image_path, target_size=(28, 28)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, target_size)
    image_inverted = 255 - image
    normalized = image_inverted / 255.0
    return normalized.reshape(28, 28, 1), image, image_inverted

# تعریف مدل با کد پیشنهادی تو (با چند تغییر برای input_shape و optimizer)
def build_cnn_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                                     kernel_initializer='he_uniform',
                                     padding='same', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                                     kernel_initializer='he_uniform',
                                     padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu',
                                    kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # استفاده از Adam به جای SGD برای بهبود آموزش (مطابق کد شما)
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["sparse_categorical_accuracy"])
    return model

# آموزش مدل روی دیتاست MNIST
def train_model(model):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = np.expand_dims(x_train, axis=-1) / 255.0
    x_test = np.expand_dims(x_test, axis=-1) / 255.0

    model.summary()
    history = model.fit(x_train, y_train, epochs=15, validation_split=0.1)
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    return model, history

# پیش‌بینی روی تصویر
def predict_image(model, image_array):
    image_input = np.expand_dims(image_array, axis=0)
    prediction = model.predict(image_input)
    predicted_class = np.argmax(prediction)
    print(f"🔍 کلاس پیش‌بینی شده: {predicted_class}")
    return predicted_class

if __name__ == "__main__":
    url = "https://raw.githubusercontent.com/maryambeheshti/Handwritten-Digit-CNN/main/2.png"
    image_path = "digit_image.png"

    download_image_from_github(url, image_path)
    image_preprocessed, img_orig, img_inverted = preprocess_image(image_path)

    model = build_cnn_model()
    model, history = train_model(model)

    predict_image(model, image_preprocessed)


✅ تصویر دانلود شد.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 813,802 (3.10 MB)

 Trainable params: 813,802 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 114s 66ms/step - loss: 0.2100 - sparse_categorical_accuracy: 0.9333 - val_loss: 0.0498 - val_sparse_categorical_accuracy: 0.9867
Epoch 2/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 138s 64ms/step - loss: 0.0360 - sparse_categorical_accuracy: 0.9891 - val_loss: 0.0406 - val_sparse_categorical_accuracy: 0.9890
Epoch 3/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 142s 64ms/step - loss: 0.0212 - sparse_categorical_accuracy: 0.9930 - val_loss: 0.0441 - val_sparse_categorical_accuracy: 0.9888
Epoch 4/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 144s 65ms/step - loss: 0.0133 - sparse_categorical_accuracy: 0.9958 - val_loss: 0.0459 - val_sparse_categorical_accuracy: 0.9892
Epoch 5/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 143s 66ms/step - loss: 0.0098 - sparse_categorical_accuracy: 0.9971 - val_loss: 0.0411 - val_sparse_categorical_accuracy: 0.9902
Epoch 6/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 141s 65ms/step - loss: 0.0072 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.0440 - val_sparse_cat